In [1]:
import glob
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import xml.etree.ElementTree as ET
import xmltodict
import torch
import torchvision
from torchvision import transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [5]:
def read_pascal_voc(file_path):
    folder = os.path.dirname(file_path)
    with open(file_path, "rb") as f:
        data = f.read()
    data = data.decode("utf-8")
    data = xmltodict.parse(data)

    annotation = data["annotation"]

    folder = annotation.get("folder", "")
    filename = annotation["filename"]
    size = annotation["size"]
    width = size["width"]
    height = size["height"]

 
    objects = []
    object_data = annotation.get("object", [])
    if not isinstance(object_data, list):
        object_data = [object_data]

    for obj in object_data:
        name = obj["name"]
        truncated = obj["truncated"]
        occluded = obj.get("occluded", 0)
        difficult = obj["difficult"]
        bndbox = obj["bndbox"]
        xmin = bndbox["xmin"]
        ymin = bndbox["ymin"]
        xmax = bndbox["xmax"]
        ymax = bndbox["ymax"]

        objects.append({
            "name": name,
            "truncated": truncated,
            "occluded": occluded,
            "difficult": difficult,
            "xmin": xmin,
            "ymin": ymin,
            "xmax": xmax,
            "ymax": ymax
        })

    data = {
        "folder": folder,
        "filename": filename,
        "width": width,
        "height": height,
    }

    #menambahkan attributes object menjadi kolom
    for obj_attr in objects[0].keys():
        data[obj_attr] = [obj[obj_attr] for obj in objects]

    return data
#paths
folder_paths = ["rupiah_banknotes_pascalvoc/Annotations/rupiah-banknotes/1000/",
                "rupiah_banknotes_pascalvoc/Annotations/rupiah-banknotes/10000/",
                "rupiah_banknotes_pascalvoc/Annotations/rupiah-banknotes/100000/",
                "rupiah_banknotes_pascalvoc/Annotations/rupiah-banknotes/2000/",
                "rupiah_banknotes_pascalvoc/Annotations/rupiah-banknotes/20000/",
                "rupiah_banknotes_pascalvoc/Annotations/rupiah-banknotes/5000/",
                "rupiah_banknotes_pascalvoc/Annotations/rupiah-banknotes/50000/",
                "rupiah_banknotes_pascalvoc/Annotations/rupiah-banknotes/75000/"]

#Membaca semua file xml dan dirubah menjadi df
data_list = []
for folder_path in folder_paths:
    xml_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".xml")]
    for xml_file in xml_files:
        data = read_pascal_voc(xml_file)
        data["folder"] = folder_path 
        data_list.append(data)

df = pd.DataFrame(data_list)
for col in df.columns:
    if isinstance(df[col][0], list):
        df[col] = df[col].explode().reset_index(drop=True)
df

,folder,filename,width,height,name,truncated,occluded,difficult,xmin,ymin,xmax,ymax
0,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/1000/0.jpg,224,224,1000,0,0,0,10.87,31.92,219.21,148.48
1,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/1000/1.jpg,224,224,1000,0,0,0,14.78,34.53,216.16,147.18
2,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/1000/1011.jpg,224,224,1000,0,0,0,0.86,50.19,214.42,165.45
3,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/1000/1027.jpg,224,224,1000,0,0,0,19.56,58.88,208.33,158.92
4,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/1000/1033.jpg,224,224,1000,0,0,0,0.43,52.36,213.99,168.93
...,...,...,...,...,...,...,...,...,...,...,...,...
745,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/75000/93.jpg,224,224,75000,0,0,0,7.69,69.46,222.13,184.72
746,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/75000/977.jpg,224,224,75000,0,0,0,5.08,50.75,218.65,154.71
747,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/75000/981.jpg,224,224,75000,0,0,0,7.26,52.06,224.0,151.23
748,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/75000/991.jpg,224,224,75000,0,0,0,2.48,62.5,224.0,172.11


In [7]:
# Membagi data menjadi training, validation, dan test set
from sklearn.model_selection import train_test_split

#train and test ratio
train_ratio = 0.8
test_ratio = 0.2

#Membagi data
df_train, df_test = train_test_split(df, test_size=test_ratio, random_state=42)

print("training:", len(df_train))
print("test:", len(df_test))

df_train.to_csv("train.csv", index=False)
df_test.to_csv("test.csv", index=False)

training: 600
test: 150


In [8]:
df_train

,folder,filename,width,height,name,truncated,occluded,difficult,xmin,ymin,xmax,ymax
595,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/5000/937.jpg,224,224,5000,0,0,0,9.0,66.85,220.82,156.45
131,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/10000/223.jpg,224,224,10000,0,0,0,7.69,46.84,215.17,149.92
44,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/1000/402.jpg,224,224,1000,0,0,0,11.74,24.96,209.64,148.05
70,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/1000/607.jpg,224,224,1000,0,0,0,7.6,43.27,216.82,162.88
672,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/50000/678.jpg,224,224,50000,0,0,0,1.17,45.97,223.87,144.27
...,...,...,...,...,...,...,...,...,...,...,...,...
71,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/1000/627.jpg,224,224,1000,0,0,0,12.61,46.71,212.68,158.92
106,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/10000/1006.jpg,224,224,10000,0,0,0,9.43,66.41,222.13,159.06
270,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/100000/655.jpg,224,224,100000,0,0,0,8.56,52.49,224.0,156.01
435,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/20000/361.jpg,224,224,20000,0,0,0,8.56,56.41,224.0,163.84


In [9]:
df_test

,folder,filename,width,height,name,truncated,occluded,difficult,xmin,ymin,xmax,ymax
506,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/5000/1056.jpg,224,224,5000,0,0,0,8.13,43.36,216.04,148.62
357,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/2000/546.jpg,224,224,2000,0,0,0,7.26,60.76,220.39,177.33
133,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/10000/260.jpg,224,224,10000,0,0,0,12.48,45.97,214.73,137.31
250,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/100000/453.jpg,224,224,100000,0,0,0,3.78,46.41,223.43,146.44
299,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/100000/991.jpg,224,224,100000,0,0,0,3.35,55.97,220.82,157.75
...,...,...,...,...,...,...,...,...,...,...,...,...
462,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/20000/612 (2).jpg,224,224,20000,0,0,0,8.56,58.58,216.47,159.06
90,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/1000/89.jpg,224,224,1000,0,0,0,10.87,56.71,217.9,178.5
393,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/2000/923.jpg,224,224,2000,0,0,0,5.52,39.88,221.26,157.32
685,rupiah_banknotes_pascalvoc/Annotations/rupiah-...,rupiah-banknotes/50000/837.jpg,224,224,50000,0,0,0,9.43,69.46,219.52,167.32


In [44]:
from PIL import Image

data_train = []
for index, row in df_train.iterrows():
    image = Image.open(row['filename'])
    mage = torch.from_numpy(np.array(image)).permute(2, 0, 1).contiguous()
    data_train.append({'image': image, 'bboxes': [[row['xmin'], row['ymin'], row['xmax'], row['ymax']]], 'labels': [row['name']]})

data_test = []
for index, row in df_test.iterrows():
    image = Image.open(row['filename'])
    mage = torch.from_numpy(np.array(image)).permute(2, 0, 1).contiguous()
    data_test.append({'image': image, 'bboxes': [[row['xmin'], row['ymin'], row['xmax'], row['ymax']]], 'labels': [row['name']]})

In [45]:
data_train

[{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
  'bboxes': [['9.0', '66.85', '220.82', '156.45']],
  'labels': ['5000']},
 {'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
  'bboxes': [['7.69', '46.84', '215.17', '149.92']],
  'labels': ['10000']},
 {'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
  'bboxes': [['11.74', '24.96', '209.64', '148.05']],
  'labels': ['1000']},
 {'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
  'bboxes': [['7.6', '43.27', '216.82', '162.88']],
  'labels': ['1000']},
 {'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
  'bboxes': [['1.17', '45.97', '223.87', '144.27']],
  'labels': ['50000']},
 {'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
  'bboxes': [['11.17', '46.41', '216.47', '146.44']],
  'labels': ['100000']},
 {'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224>,
  'bboxe

In [46]:
#Buat one-hot encoding untuk label
label_map = {label: idx for idx, label in enumerate(set(df_train['name']))}
num_classes = len(label_map)

def collate_fn(batch):
    return tuple(zip(*batch))[:-1]

In [47]:
#Buat dataloader
from torch.utils.data import DataLoader
train_loader = DataLoader(data_train, batch_size=16, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(data_test, batch_size=16, shuffle=False, collate_fn=collate_fn)

In [48]:
#Load model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

C:\Users\M Zuhdi Hanif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\M Zuhdi Hanif\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [49]:
#Training
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

model.to(device)
for epochs in range(100):
    epoch_loss = 0
    for data in train_loader:
        imgs, targets = data
        imgs = [img.to(device) for img in imgs]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(imgs, targets)
        loss = sum(v for v in loss_dict.values())
        epoch_loss += loss.cpu().detach().numpy()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

AttributeError: 'str' object has no attribute 'to'